<a href="https://colab.research.google.com/github/nikhil-mathews/MastersPr_Predicting-Human-Pathogen-PPIs-using-Natural-Language-Processing-methods/blob/main/4.Final_model/5FCV_final_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
#Google colab does not have pickle
try:
  import pickle5 as pickle
except:
  !pip install pickle5
  import pickle5 as pickle
import os
import seaborn as sns

import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, GlobalMaxPooling1D,Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding,Concatenate
from keras.models import Model,load_model
from sklearn.metrics import roc_auc_score,confusion_matrix,roc_curve, auc
from numpy import random
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout
from keras.optimizers import Adam
from keras.utils.vis_utils import plot_model
from keras.metrics import AUC

import sys
sys.path.insert(0,'/content/drive/MyDrive/ML_Data/')
import functions as f

     |████████████████████████████████| 133kB 13.4MB/s 
  Created wheel for pickle5: filename=pickle5-0.0.11-cp37-cp37m-linux_x86_64.whl size=219251 sha256=0ed2d4ee6d1b98985a5e2e268de3aa39474b4199aacf14dce72f80e17bdfc1d3
  Stored in directory: /root/.cache/pip/wheels/a6/90/95/f889ca4aa8b0e0c7f21c8470b6f5d6032f0390a3a141a9a3bd
Successfully built pickle5


In [ ]:
def load_data(D=1,randomize=False):
    try:
        with open('/content/drive/MyDrive/ML_Data/df_train_'+str(D)+'D.pickle', 'rb') as handle:
            df_train = pickle.load(handle)
    except:
        df_train = pd.read_pickle("C:/Users/nik00/py/proj/hyppi-train.pkl")
    try:
        with open('/content/drive/MyDrive/ML_Data/df_test_'+str(D)+'D.pickle', 'rb') as handle:
            df_test = pickle.load(handle)
    except:
        df_test = pd.read_pickle("C:/Users/nik00/py/proj/hyppi-independent.pkl")
    if randomize:
        return shuff_together(df_train,df_test)
    else:
        return df_train,df_test

df_train,df_test = load_data(5)
print('The data used will be:')
df_train

The data used will be:


,Human,Yersinia,Joined,label
0,"[MKDKQ, KDKQK, DKQKK, KQKKK, QKKKK, KKKKE, KKK...","[MAKAS, AKASR, KASRH, ASRHN, SRHNL, RHNLS, HNL...","[MKDKQ, KDKQK, DKQKK, KQKKK, QKKKK, KKKKE, KKK...",1
1,"[MSWSG, SWSGL, WSGLL, SGLLH, GLLHG, LLHGL, LHG...","[MQHVT, QHVTG, HVTGS, VTGSK, TGSKR, GSKRR, SKR...","[MSWSG, SWSGL, WSGLL, SGLLH, GLLHG, LLHGL, LHG...",0
2,"[MSLFD, SLFDL, LFDLF, FDLFR, DLFRG, LFRGF, FRG...","[MAELP, AELPA, ELPAK, LPAKR, PAKRR, AKRRF, KRR...","[MSLFD, SLFDL, LFDLF, FDLFR, DLFRG, LFRGF, FRG...",1
3,"[MAVAP, AVAPR, VAPRL, APRLF, PRLFG, RLFGG, LFG...","[MRIFA, RIFAI, IFAIS, FAISC, AISCS, ISCSS, SCS...","[MAVAP, AVAPR, VAPRL, APRLF, PRLFG, RLFGG, LFG...",0
4,"[MSTIQ, STIQS, TIQSE, IQSET, QSETD, SETDC, ETD...","[MSYAF, SYAFP, YAFPG, AFPGT, FPGTF, PGTFP, GTF...","[MSTIQ, STIQS, TIQSE, IQSET, QSETD, SETDC, ETD...",0
...,...,...,...,...
6265,"[MSYFG, SYFGE, YFGEH, FGEHF, GEHFW, EHFWG, HFW...","[MITTD, ITTDG, TTDGN, TDGNS, DGNSA, GNSAV, NSA...","[MSYFG, SYFGE, YFGEH, FGEHF, GEHFW, EHFWG, HFW...",1
6266,"[MTVGK, TVGKS, VGKSS, GKSSK, KSSKM, SSKML, SKM...","[MSQPP, SQPPF, QPPFW, PPFWQ, PFWQQ, FWQQK, WQQ...","[MTVGK, TVGKS, VGKSS, GKSSK, KSSKM, SSKML, SKM...",1
6267,"[MNNLS, NNLSF, NLSFS, LSFSE, SFSEL, FSELC, SEL...","[MSEDR, SEDRH, EDRHQ, DRHQQ, RHQQR, HQQRQ, QQR...","[MNNLS, NNLSF, NLSFS, LSFSE, SFSEL, FSELC, SEL...",0
6268,"[MAPEI, APEIN, PEINL, EINLP, INLPG, NLPGP, LPG...","[MKNLS, KNLSF, NLSFV, LSFVA, SFVAG, FVAGL, VAG...","[MAPEI, APEIN, PEINL, EINLP, INLPG, NLPGP, LPG...",1


### Stratified K-Fold CV

In [ ]:
# skf = StratifiedKFold(n_splits=5,shuffle=True)
# X = df_train[['Human','Yersinia']]
# y = df_train[['label']]
# skf.get_n_splits(X, y)
# trains = []
# tests = []
# for train_index, test_index in skf.split(X, y):
#   trains.append(df_train.iloc[train_index].reset_index(drop=True))
#   tests.append(df_train.iloc[test_index].reset_index(drop=True))

# cv_data = (trains,tests)
# f.save(cv_data,'5Fcrossval_data')

five_trains,five_tests = f.load('5Fcrossval_data')

In [ ]:
EMBEDDING_DIM_5D = 25
BATCH_SIZE = 16
EPOCHS = 1
DROP = 0.5
MAX_SEQUENCE_LENGTH_5D_J = 2000
MAX_SEQUENCE_LENGTH_5D_dIP = 1000
num_words_5D_J = 1000000
num_words_5D = 500000
scores = []

for i in range(5):
  df_train,df_test = five_trains[i],five_tests[i]
  print("Fold",i+1)
  trains = f.preprocess(df_train,saveTokrs=False)
  tests = f.preprocess(df_test,saveTokrs=False)
  x1_join = f.transf_model(MAX_SEQUENCE_LENGTH_5D_J,num_words_5D_J,5,0.9)
  x2_join = f.transf_model(MAX_SEQUENCE_LENGTH_5D_J,num_words_5D_J,5,0.9)
  x3_join = f.transf_model(MAX_SEQUENCE_LENGTH_5D_J,num_words_5D_J,5,0.9)

  x1_doubleip = f.conv_model(MAX_SEQUENCE_LENGTH_5D_dIP,EMBEDDING_DIM_5D,num_words_5D,DROP)
  x2_doubleip = f.conv_model(MAX_SEQUENCE_LENGTH_5D_dIP,EMBEDDING_DIM_5D,num_words_5D,DROP)
  x3_doubleip = f.conv_model(MAX_SEQUENCE_LENGTH_5D_dIP,EMBEDDING_DIM_5D,num_words_5D,DROP)
  x4_doubleip = f.conv_model(MAX_SEQUENCE_LENGTH_5D_dIP,EMBEDDING_DIM_5D,num_words_5D,DROP)
  x5_doubleip = f.conv_model(MAX_SEQUENCE_LENGTH_5D_dIP,EMBEDDING_DIM_5D,num_words_5D,DROP)
  x6_doubleip = f.conv_model(MAX_SEQUENCE_LENGTH_5D_dIP,EMBEDDING_DIM_5D,num_words_5D,DROP)

  concatenator = Concatenate(axis=1)
  x = concatenator([x1_join.output, x2_join.output, x3_join.output, x1_doubleip.output, x2_doubleip.output, x3_doubleip.output, x4_doubleip.output, x5_doubleip.output, x6_doubleip.output])
  x = Dense(256)(x)
  x = Dropout(0.5)(x)
  output = Dense(1, activation="sigmoid",name="Final")(x)
  model_final = Model(inputs=[x1_join.input, x2_join.input, x3_join.input, x1_doubleip.input, x2_doubleip.input, x3_doubleip.input, x4_doubleip.input, x5_doubleip.input, x6_doubleip.input], outputs=output)

  model_final.compile(loss='binary_crossentropy', optimizer='adam', metrics=AUC())

  model_final.fit(trains, df_train['label'].values, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(tests,df_test['label'].values))
  score = roc_auc_score(df_test['label'].values, model_final.predict(tests))
  score = round(score,3)
  scores.append(score)
  #print("Score is",score)

print("AUC by cross validation is",round(sum(scores) / len(scores),2))

Fold 1
Preprocessing...
Preprocessing...
314/314 [==============================] - 192s 589ms/step - loss: 0.6215 - auc_1: 0.7465 - val_loss: 0.4193 - val_auc_1: 0.9202
Fold 2
Preprocessing...
Preprocessing...
314/314 [==============================] - 178s 545ms/step - loss: 0.6403 - auc_2: 0.7196 - val_loss: 0.4748 - val_auc_2: 0.9003
Fold 3
Preprocessing...
Preprocessing...
314/314 [==============================] - 191s 586ms/step - loss: 0.6233 - auc_3: 0.7352 - val_loss: 0.4017 - val_auc_3: 0.9064
Fold 4
Preprocessing...
Preprocessing...
314/314 [==============================] - 176s 541ms/step - loss: 0.6434 - auc_4: 0.7259 - val_loss: 0.4040 - val_auc_4: 0.9183
Fold 5
Preprocessing...
Preprocessing...
314/314 [==============================] - 178s 543ms/step - loss: 0.6448 - auc_5: 0.7248 - val_loss: 0.4076 - val_auc_5: 0.9051
AUC by cross validation is 0.91


In [ ]:
print("AUC by 3 decimal places",round(sum(scores) / len(scores),3))

AUC by 3 decimal places 0.91
